# Graded Lab: Tool Use and Reflective Agents

In this lab, you will explore how AI agents can enhance research workflows by leveraging external tools and engaging in critical self-reflection.

### Learning Objectives

- Chain steps into a research pipeline (**search → reflection → formatting**).
- Convert natural-language output into **styled HTML** suitable for sharing.

In [1]:
import json

from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, HTML

import research_tools

load_dotenv()

CLIENT = OpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

## Using Tools

You'll use two research tools exposed in the `research_tools` module:
- **`arxiv_search_tool(query, max_results)`** – academic papers via arXiv API.
- **`tavily_search_tool(query, max_results, include_images)`** – general web search via Tavily.

In [ ]:
topic = "linear algebra"

arxiv_results = research_tools.arxiv_search_tool(topic, max_results=3)

for i, paper in enumerate(arxiv_results, 1):
    if "error" in paper:
        print(f"Error: {paper['error']}")
    else:
        print(f"Paper {i}")
        print(f"  Title     : {paper['title']}")
        print(f"  Authors   : {', '.join(paper['authors'])}")
        print(f"  Published : {paper['published']}")
        print(f"  URL       : {paper['url']}\n")

print("\nRaw Results:\n")
print(json.dumps(arxiv_results, indent=2))

In [ ]:
topic = "retrieval-augmented generation applications"

tavily_results = research_tools.tavily_search_tool(topic)
for item in tavily_results:
    print(item)

## Tool Mapping

Dictionary that maps tool names (strings) to actual Python functions.

In [ ]:
TOOL_MAPPING = {
    "tavily_search_tool": research_tools.tavily_search_tool,
    "arxiv_search_tool": research_tools.arxiv_search_tool,
}

## Exercise 1: Generate Research Report with Tools

**Goal:** Implement `generate_research_report_with_tools(prompt)`.

This function generates a detailed research report with the assistance of online tools.

In [ ]:
def generate_research_report_with_tools(prompt: str, model: str = "gpt-4o") -> str:
    """
    Generates a research report using OpenAI's tool-calling with arXiv and Tavily tools.

    Args:
        prompt (str): The user prompt.
        model (str): OpenAI model name.

    Returns:
        str: Final assistant research report text.
    """
    messages = [
        {
            "role": "system",
            "content": (
                "You are a research assistant that can search the web and arXiv to write detailed, "
                "accurate, and properly sourced research reports.\n\n"
                "Use tools when appropriate (e.g., to find scientific papers or web content).\n"
                "Cite sources whenever relevant. Do NOT omit citations for brevity.\n"
                "When possible, include full URLs (arXiv links, web sources, etc.).\n"
                "Use an academic tone, organize output into clearly labeled sections, and include "
                "inline citations or footnotes as needed.\n"
                "Do not include placeholder text such as '(citation needed)' or '(citations omitted)'."
            )
        },
        {"role": "user", "content": prompt}
    ]

    tools = [research_tools.arxiv_tool_def, research_tools.tavily_tool_def]
    max_turns = 10
    final_text = ""
    
    for _ in range(max_turns):

        ### START CODE HERE ###

        response = CLIENT.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            temperature=1,
        )

        ### END CODE HERE ###

        msg = response.choices[0].message
        messages.append(msg)

        if not msg.tool_calls:
            final_text = msg.content
            print("Final answer:")
            print(final_text)
            break

        for call in msg.tool_calls:
            tool_name = call.function.name
            args = json.loads(call.function.arguments)
            print(f"Tool: {tool_name}({args})")

            try:
                tool_func = TOOL_MAPPING[tool_name]
                result = tool_func(**args)
            except Exception as e:
                result = {"error": str(e)}

            ### START CODE HERE ###

            new_msg = {
                "role": "tool",
                "tool_call_id": call.id,
                "name": tool_name,
                "content": json.dumps(result)
            }

            ### END CODE HERE ###

            messages.append(new_msg)

    return final_text

## Exercise 2: Reflection + Rewrite

**Goal:** Implement `reflection_and_rewrite(report)`.

This function takes a report, analyzes it, generates a structured reflection, and produces an improved version.

In [ ]:
def reflection_and_rewrite(report, model: str = "gpt-4o-mini", temperature: float = 0.3) -> dict:
    """
    Generates a structured reflection AND a revised research report.
    Accepts raw text OR the messages list returned by generate_research_report_with_tools.

    Returns:
        dict with keys:
          - "reflection": structured reflection text
          - "revised_report": improved version of the input report
    """
    report = research_tools.parse_input(report)

    ### START CODE HERE ###

    user_prompt = f"""Analyze the following research report and provide a structured reflection and revised version.

Report:
{report}

Your response must be ONLY valid JSON with exactly this structure:
{{"reflection": "Your analysis covering Strengths, Limitations, Suggestions, and Opportunities", "revised_report": "Your improved version of the report"}}"""

    response = CLIENT.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an academic reviewer and editor."},
            {"role": "user", "content": user_prompt},
        ],
        temperature=temperature
    )

    ### END CODE HERE ###

    llm_output = response.choices[0].message.content.strip()

    try:
        data = json.loads(llm_output)
    except json.JSONDecodeError:
        raise Exception("The output of the LLM was not valid JSON. Adjust your prompt.")

    return {
        "reflection": str(data.get("reflection", "")).strip(),
        "revised_report": str(data.get("revised_report", "")).strip(),
    }

## Exercise 3: Convert Report to HTML

**Goal:** Implement `convert_report_to_html(report)`.

This function transforms a plain text research report into a well-structured HTML document.

In [ ]:
def convert_report_to_html(report, model: str = "gpt-4o", temperature: float = 0.5) -> str:
    """
    Converts a plaintext research report into a styled HTML page using OpenAI.
    Accepts raw text OR the messages list from the tool-calling step.
    """
    report = research_tools.parse_input(report)

    system_prompt = "You convert plaintext reports into full clean HTML documents."

    ### START CODE HERE ###

    user_prompt = f"""Convert this research report into a well-structured HTML document with proper headings, paragraphs, and clickable links. Return ONLY valid HTML.

Report:
{report}"""

    response = CLIENT.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=temperature
    )

    ### END CODE HERE ###

    html = response.choices[0].message.content.strip()

    return html

## End-to-End Pipeline

Run this cell to execute the full workflow:

1. Generate a research report (tools)
2. Reflect on the report
3. Convert the report to HTML

In [1]:
prompt_ = "Radio observations of recurrent novae"
preliminary_report = generate_research_report_with_tools(prompt_)
print("=== Research Report (preliminary) ===\n")
print(preliminary_report)

NameError: name 'generate_research_report_with_tools' is not defined

In [2]:
reflection_text = reflection_and_rewrite(preliminary_report)
print("=== Reflection on Report ===\n")
print(reflection_text['reflection'], "\n")
print("=== Revised Report ===\n")
print(reflection_text['revised_report'], "\n")

NameError: name 'reflection_and_rewrite' is not defined

In [3]:
html = convert_report_to_html(reflection_text['revised_report'])

print("=== Generated HTML (preview) ===\n")
print((html or "")[:600], "\n... [truncated]\n")

display(HTML(html))

NameError: name 'convert_report_to_html' is not defined

## Wrap-Up

You built a mini research agent that can:
- Call tools (arXiv + Tavily)
- Reflect on its own output
- Publish a clean HTML report